In [ ]:
!python --version
# Install necessary packages
!pip install --upgrade pip
!pip install pycocotools
!pip install mediapipe-model-maker

Python 3.10.12


In [ ]:
from google.colab import drive
import shutil
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Compress the Folder
folder_path = '/content/drive/My Drive/datasets'  # Change to your folder path
zip_file_path = '/content/datasets.zip'  # Save directly to /content
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f'Folder compressed to {zip_file_path}')

# Step 3: Create the /content/datasets directory if it doesn't exist
output_folder = '/content/datasets'
os.makedirs(output_folder, exist_ok=True)

# Unzip the contents into the /content/datasets folder
!unzip -o {zip_file_path} -d {output_folder}

print(f'Contents unzipped to {output_folder}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder compressed to /content/datasets.zip
Archive:  /content/datasets.zip
   creating: /content/datasets/test/
   creating: /content/datasets/train/
   creating: /content/datasets/valid/
   creating: /content/datasets/test/images/
  inflating: /content/datasets/test/labels.json  
  inflating: /content/datasets/test/images/06_05051_png.rf.67738268d0cfb761b01d44e1b9c467ec.jpg  
  inflating: /content/datasets/test/images/06_04881_png.rf.9eb174bc9602f59ccf5d120fb2d1e4e7.jpg  
  inflating: /content/datasets/test/images/07_02561_png.rf.88d97652520f43cdae4570cb4e683b0e.jpg  
  inflating: /content/datasets/test/images/06_08681_png.rf.405589c2bd61facefc918fcd187c18c6.jpg  
  inflating: /content/datasets/test/images/06_04871_png.rf.ee2e82fe9d903006fbf7df5b7bae9b4b.jpg  
  inflating: /content/datasets/test/images/06_05021_png.rf.1fd7be0a8886d60ec3ee2db420f29cf8.jpg  
 

In [ ]:
# Import required libraries
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import tensorflow as tf
from mediapipe_model_maker import object_detector
from google.colab import files
# Check TensorFlow version
assert tf.__version__.startswith('2')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Define dataset paths
dataset_path = '/content/datasets'
train_dataset_path = os.path.join(dataset_path, 'train')
validation_dataset_path = os.path.join(dataset_path, 'valid')

# Load COCO dataset
coco_gt = COCO(os.path.join(train_dataset_path, "labels.json"))  # Path to your COCO annotations
image_ids = coco_gt.getImgIds()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
# Load training and validation datasets
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/valid")

In [ ]:
# Define model specifications and hyperparameters
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
# Train the model
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


70/70 [==============================] - 271s 3s/step - total_loss: 4.1070 - cls_loss: 3.5174 - box_loss: 0.0105 - model_loss: 4.0442 - val_total_loss: 1.6231 - val_cls_loss: 1.1538 - val_box_loss: 0.0081 - val_model_loss: 1.5602
Epoch 2/30
70/70 [==============================] - 209s 3s/step - total_loss: 1.3737 - cls_loss: 1.0115 - box_loss: 0.0060 - model_loss: 1.3108 - val_total_loss: 1.1829 - val_cls_loss: 0.8440 - val_box_loss: 0.0055 - val_model_loss: 1.1200
Epoch 3/30
70/70 [==============================] - 209s 3s/step - total_loss: 0.8828 - cls_loss: 0.6420 - box_loss: 0.0036 - model_loss: 0.8199 - val_total_loss: 0.7007 - val_cls_loss: 0.4498 - val_box_loss: 0.0038 - val_model_loss: 0.6378
Epoch 4/30
70/70 [==============================] - 211s 3s/step - total_loss: 0.5881 - cls_loss: 0.3943 - box_loss: 0.0026 - model_loss: 0.5252 - val_total_loss: 0.6165 - val_cls_loss: 0.3767 - val_box_loss: 0.0035 - val_model_loss: 0.5535
Epoch 5/30
70/70 [=============================

In [ ]:
# Evaluate the model
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

24/24 [==============================] - 9s 303ms/step - total_loss: 0.4090 - cls_loss: 0.2535 - box_loss: 0.0019 - model_loss: 0.3460
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.35s).
Accumulating evaluation results...
DONE (t=0.10s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.684
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.999
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.824
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.695
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.634
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.723
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.729
 Average Recall     (AR) @[

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


141/141 [==============================] - 321s 2s/step - total_loss: 1.3948 - cls_loss: 0.8373 - box_loss: 0.0112 - model_loss: 1.3948 - val_total_loss: 1.3364 - val_cls_loss: 0.9850 - val_box_loss: 0.0070 - val_model_loss: 1.3364
Epoch 2/10
141/141 [==============================] - 238s 2s/step - total_loss: 0.6854 - cls_loss: 0.4107 - box_loss: 0.0055 - model_loss: 0.6854 - val_total_loss: 0.7075 - val_cls_loss: 0.4486 - val_box_loss: 0.0052 - val_model_loss: 0.7075
Epoch 3/10
141/141 [==============================] - 235s 2s/step - total_loss: 0.5311 - cls_loss: 0.3216 - box_loss: 0.0042 - model_loss: 0.5311 - val_total_loss: 0.5488 - val_cls_loss: 0.3162 - val_box_loss: 0.0047 - val_model_loss: 0.5488
Epoch 4/10
141/141 [==============================] - 234s 2s/step - total_loss: 0.4927 - cls_loss: 0.2990 - box_loss: 0.0039 - model_loss: 0.4927 - val_total_loss: 0.5392 - val_cls_loss: 0.3085 - val_box_loss: 0.0046 - val_model_loss: 0.5392
Epoch 5/10
141/141 [===================

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_2 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

141/141 [==============================] - 323s 2s/step - total_loss: 1.2468 - cls_loss: 0.7350 - box_loss: 0.0102 - model_loss: 1.2468 - val_total_loss: 0.8688 - val_cls_loss: 0.5278 - val_box_loss: 0.0068 - val_model_loss: 0.8688
Epoch 2/15
141/141 [==============================] - 238s 2s/step - total_loss: 0.6805 - cls_loss: 0.3937 - box_loss: 0.0057 - model_loss: 0.6805 - val_total_loss: 0.5891 - val_cls_loss: 0.3207 - val_box_loss: 0.0054 - val_model_loss: 0.5891
Epoch 3/15
141/141 [==============================] - 239s 2s/step - total_loss: 0.5489 - cls_loss: 0.3190 - box_loss: 0.0046 - model_loss: 0.5489 - val_total_loss: 0.5668 - val_cls_loss: 0.3080 - val_box_loss: 0.0052 - val_model_loss: 0.5668
Epoch 4/15
141/141 [==============================] - 236s 2s/step - total_loss: 0.5212 - cls_loss: 0.3092 - box_loss: 0.0042 - model_loss: 0.5212 - val_total_loss: 0.5588 - val_cls_loss: 0.3070 - val_box_loss: 0.0050 - val_model_loss: 0.5588
Epoch 5/15
141/141 [===================

In [ ]:
model.export_model('model_int8_qat.tflite')
!ls -lh exported_model
files.download('exported_model/model_int8_qat.tflite')

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")